In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *
from process import * 
from tqdm.notebook import tqdm

In [2]:
# データ保存先
data_folda = '../data/'

* masterからデータを追加

In [3]:

### master data から加えるデータを定義
add_list=['name',"prefecture_name",'head_branch','opened_on', 'created_at','genre_first_name', 
              'genre_second_name','genre_third_name','northlatitude', 'eastlongitude',
              "net_reservation_flg",'price_range_lunch_owner','price_range_dinner_owner', 
              'price_range_lunch_user','price_range_dinner_user',
              # 20240312追加
              'all_photo_count','photo_food_count','photo_drink_count',
              'photo_interior_count','photo_exterior_count','official_info_flg',
              'max_visit_month'
              ]

# make_timeseries_data.ipynb で作成したデータを読み込む
pseudo_time_series=pd.read_pickle(data_folda+'skeleton_train.pkl')

# 直近三カ月の閉店率を計算して保存
# pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-08-31']
# close_ratios=calculate_close_ratio(pseudo_time_series, data_folda)
# close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 三カ月の閉店率を読み込み
close_ratios=pd.read_pickle(data_folda+'close_ratios.pkl')

# 2023-01-01からの閉店を予測する。base_date が 2023-01-01の一日前以後が対象
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
# base_date が 2023年1月1日以降のデータを抽出 (closed_date がそれ以降の閉店店舗しかクロールしていないため)
pseudo_time_series = add_master_data(pseudo_time_series, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series=pd.merge(pseudo_time_series,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series.to_pickle(data_folda+'train_with_master.pkl')

pseudo_time_series_test3=pd.read_pickle(data_folda+'skeleton_test_3month.pkl')
pseudo_time_series_test3 = add_master_data(pseudo_time_series_test3, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test3,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master.pkl')

pseudo_time_series_test6=pd.read_pickle(data_folda+'skeleton_test_6month.pkl')
pseudo_time_series_test6 = add_master_data(pseudo_time_series_test6, data_folda, add_list, "restaurant_data.csv")
pseudo_time_series_test3=pd.merge(pseudo_time_series_test6,close_ratios,on=["base_date","genre_first_name"],how="left")
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master.pkl')

In [6]:
pseudo_time_series.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10503768 entries, 0 to 10503767
Data columns (total 33 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   restaurant_id             object        
 1   original_id               int64         
 2   closed_date               datetime64[ns]
 3   base_date                 datetime64[ns]
 4   target3                   int64         
 5   target6                   int64         
 6   eval_period               bool          
 7   id                        int64         
 8   name                      object        
 9   prefecture_name           object        
 10  head_branch               object        
 11  opened_on                 datetime64[ns]
 12  created_at                object        
 13  genre_first_name          object        
 14  genre_second_name         object        
 15  genre_third_name          object        
 16  northlatitude             float64       
 17  eastlo

* V_GOOGLE からデータを追加

In [13]:
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series = add_v_google(pseudo_time_series, data_folda, add_list)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master.pkl')
add_list=["restaurant_id","overview.aggregateRating","overview.reviewCount","general_rank"]
pseudo_time_series_test = add_v_google(pseudo_time_series_test, data_folda, add_list)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_RETTY からデータを追加

In [3]:
#追加する変数をリストで定義
add_list=["restaurant_id",'infos.isOfficial', 
       'infos.familiar.users.count', 'infos.familiar.stars',
       'general_rank_RETTY', 
       'infos.onlineReservation_RETTY', 'infos.access.transferTime1','infos.access.transferTime2','infos.access.transferTime3',
       'infos.numberOfSeats.value', 'infos.counterSeats.value',
       'infos.privateRoom.value', 'infos.updateInfo.firstReviewDate',
       'infos.wantToGo', 'infos.updateInfo.lastReviewDate',
       'infos.went', 'infos.rateByWent', 'infos.excellent', 'infos.coupon',
       'infos.photoCount.Photograph', 'infos.photoCount.Cooking',
       'infos.photoCount.Interior', 'infos.photoCount.Exterior',
       'infos.photoCount.Menu', 'infos.reviewCount.Review',
       'infos.reviewCount.Lunch', 'infos.reviewCount.Dinner',
       'menus.course_list_count',
       'menus.dishes_list_count', 'menus.lunch_list_count',
       'menus.drink_list_count', 'menus.takeout_list_count']
# 'infos.familiar.users.genge','infos.reserveEntireStore',
 
pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google.pkl')         
pseudo_time_series = add_v_retty(pseudo_time_series, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google.pkl')
pseudo_time_series_test = add_v_retty(pseudo_time_series_test, data_folda, add_list, master_name="V_RETTY",read_from_snowflake=False)
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty.pkl')

del pseudo_time_series, pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* HOTPEPPER からデータを追加

In [3]:
add_list=['restaurant_id','general_rank_HOTPEPPER','infos.aggregateRating','infos.ratingReview',
          'infos.satisfaction.percentage1','infos.satisfaction.percentage2','infos.satisfaction.percentage3',
          'infos.couponCount','infos.onlineReservation_HOTPEPPER','infos.ownerRegistration','infos.ticket',
          'photos.allCount','photos.appearanceCount','photos.drinkCount','photos.foodCount','photos.otherCount',
          'photos.postCount','menus.courseListCount']


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty.pkl')
pseudo_time_series=pseudo_time_series[pseudo_time_series['base_date']>='2022-12-31']
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hotpepper(pseudo_time_series, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty.pkl')
pseudo_time_series_test3= reduce_mem_usage(pseudo_time_series_test3)
pseudo_time_series_test3 = add_v_hotpepper(pseudo_time_series_test3, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty.pkl')
pseudo_time_series_test6= reduce_mem_usage(pseudo_time_series_test6)
pseudo_time_series_test6 = add_v_hotpepper(pseudo_time_series_test6, data_folda, add_list, master_name="V_HOTPEPPER",read_from_snowflake=False) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper.pkl')

del pseudo_time_series, pseudo_time_series_test3, pseudo_time_series_test6

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_STRUCTURED からデータを取得

In [4]:
add_list = '"restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount", \
    "aggregateRating.ratingValue", "aggregateRating.reviewCount", \
    "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went", \
    "interactionCount.wanttogo","paymentAccepted"'


pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper.pkl')
pseudo_time_series = add_v_structured(pseudo_time_series, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series

pseudo_time_series_test3 = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper.pkl')
pseudo_time_series_test3 = add_v_structured(pseudo_time_series_test3, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test3.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured.pkl')

pseudo_time_series_test6 = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper.pkl')
pseudo_time_series_test6 = add_v_structured(pseudo_time_series_test6, data_folda, add_list, master_name="V_STRUCTURED",read_from_snowflake=True) 
pseudo_time_series_test6.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured.pkl')

del pseudo_time_series_test3, pseudo_time_series_test6

select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
demae-can.com
twitter.com
www.google.co.jp
www.tripadvisor.jp
sarah30.com
paypaygourmet.yahoo.co.jp
hitosara.com
place.line.me
www.ozmall.co.jp
4travel.jp
rtrp.jp
www.instagram.com
www.ekiten.jp
www.navitime.co.jp
www.jalan.net
r.gnavi.co.jp
www.facebook.com
retty.me
restaurant.ikyu.com
tabelog.com
www.ubereats.com
gourmet.aumo.jp
loco.yahoo.co.jp
autoreserve.com
www.hotpepper.jp
ramendb.supleks.jp
tokyo-calendar.jp
select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
demae-can.com
twitter.com
www.google.co.jp
www.tripadvisor.jp
sarah30.com
paypaygourmet.yahoo.co.jp
hitosara.com
place.line.me
www.ozmall.co.jp
4travel.jp
rtrp.jp
www.instagram.com
www.ekiten.jp
www.navitime.co.jp
www.jalan.net
r.gnavi.co.jp
www.facebook.com
retty.me
restaurant.ikyu.com
tabelog.com
www.ubereats.com
gourmet.aumo.jp
loco.yahoo.co.jp
autoreserve.com
www.hotpepper.jp
ramendb.supleks.jp
tokyo-calendar.jp
select "restaurant_id","url","name_rank","address_rank","telephone_rank","aggregateRating.ratingCount",     "aggregateRating.ratingValue", "aggregateRating.reviewCount",     "aggregateRating.bestRating", "aggregateRating.worstRating", "interactionCount.went",     "interactionCount.wanttogo","paymentAccepted" from KRAKEN_DG.KRAKEN_USER.V_STRUCTURED


/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

select "name","hostname" from KRAKEN_DG.KRAKEN_USER.V_MEDIA_SITE_LIST
demae-can.com
twitter.com
www.google.co.jp
www.tripadvisor.jp
sarah30.com
paypaygourmet.yahoo.co.jp
hitosara.com
place.line.me
www.ozmall.co.jp
4travel.jp
rtrp.jp
www.instagram.com
www.ekiten.jp
www.navitime.co.jp
www.jalan.net
r.gnavi.co.jp
www.facebook.com
retty.me
restaurant.ikyu.com
tabelog.com
www.ubereats.com
gourmet.aumo.jp
loco.yahoo.co.jp
autoreserve.com
www.hotpepper.jp
ramendb.supleks.jp
tokyo-calendar.jp


* V_TRIPADVISOR から特徴量を追加

In [3]:
add_list=["restaurant_id",'general_rank_TRIPADVISOR','infos.aggregateRating_TRIPADVISOR',"infos.reviewCount","infos.qa_count"]

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series = reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_tripadvisor(pseudo_time_series, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured_tripadvisor.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured.pkl')
pseudo_time_series_test = reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_tripadvisor(pseudo_time_series_test, data_folda, add_list, master_name="V_TRIPADVISOR",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured_tripadvisor.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

* V_HITOSARA から特徴量を追加

In [4]:
# hitosara premium を追加

add_list=["restaurant_id","general_rank_HITOSARA","infos.premium","infos.onlineReservation_HITOSARA"]

pseudo_time_series = pd.read_pickle(data_folda+'train_with_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series= reduce_mem_usage(pseudo_time_series)
pseudo_time_series = add_v_hitosara(pseudo_time_series, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False)
pseudo_time_series.to_pickle(data_folda+'train_with_master_structured_main_medias.pkl')

del pseudo_time_series

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_3month_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'test3_with_master_structured_main_medias.pkl')

pseudo_time_series_test = pd.read_pickle(data_folda+'skeleton_test_6month_master_google_retty_hotpepper_structured_tripadvisor.pkl')
pseudo_time_series_test= reduce_mem_usage(pseudo_time_series_test)
pseudo_time_series_test = add_v_hitosara(pseudo_time_series_test, data_folda, add_list, master_name="V_HITOSARA",read_from_snowflake=False) 
pseudo_time_series_test.to_pickle(data_folda+'test6_with_master_structured_main_medias.pkl')

del pseudo_time_series_test

/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["n_rank_count"] = df["telephone_rank"].isin(["N"]).astype(int) + df["address_rank"].isin(["N"]).astype(int) + df["name_rank"].isin(["N"]).astype(int)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["denominator"] = np.select(condition, choice, default=-1)
/home/ec2-user/dglab-ai-scoring_insitestream/pg/./utils/rank.py:78: Se

In [3]:
pseudo_time_series=pd.read_pickle(data_folda+'train_with_master_structured_main_medias.pkl')
pseudo_time_series.info()
pseudo_time_series_filter=pseudo_time_series.drop_duplicates(subset=['restaurant_id'])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10503768 entries, 0 to 10503767
Columns: 114 entries, restaurant_id to infos.onlineReservation_HITOSARA
dtypes: bool(1), category(3), datetime64[ns](5), float32(47), float64(4), int16(3), int32(5), int8(4), object(42)
memory usage: 6.2+ GB


In [132]:
pseudo_time_series_filter.shape

(875314, 114)

In [62]:
# pseudo_time_series_filterからカラム名に general_rank もの含むを抽出
pseudo_time_series_filter_columns = pseudo_time_series_filter.columns
general_rank_columns = [column for column in pseudo_time_series_filter_columns if 'general_rank' in column]
general_rank_columns

['general_rank_GOOGLE',
 'general_rank_RETTY',
 'general_rank_HOTPEPPER',
 'general_rank_STRUCTURED',
 'general_rank_TRIPADVISOR',
 'general_rank_HITOSARA']

In [ ]:
# pseudo_time_series_filter[general_rank_columns]をプロット
# 横軸を S,A,B,L,F,N,NaNの分布
# 一つのキャンバスにプロット

fig, ax = plt.subplots()

for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    ax.hist(pseudo_time_series_filter[column])

plt.show()
"""
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].unique())
    # 前のプロットを上書きする
    pseudo_time_series_filter[column].value_counts().plot(kind='bar',title=column,ax=plt.gca())
    plt.show()
"""


In [41]:
skeleton_train["general_rank_HITOSARA"]

0           S
1           S
2           S
3           S
4           S
           ..
52518835    S
52518836    S
52518837    S
52518838    S
52518839    S
Name: general_rank_HITOSARA, Length: 52518840, dtype: object

In [11]:
skeleton_train=pd.read_pickle(data_folda+'skeleton_train.pkl')
skeleton_train=pd.merge(skeleton_train,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_train.to_pickle(data_folda+'skeleton_train_with_general_rank.pkl')

In [44]:
skeleton_test=pd.read_pickle(data_folda+'skeleton_test.pkl')
skeleton_test=pd.merge(skeleton_test,pseudo_time_series_filter[["restaurant_id"]+general_rank_columns],on="restaurant_id",how="left")
skeleton_test.to_pickle(data_folda+'skeleton_test_with_general_rank.pkl')

In [55]:
g_filter=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter[g_filter["general_rank_RETTY"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HOTPEPPER"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_HITOSARA"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_TRIPADVISOR"].isin(["A","B","S","L","N","F"])]
g_filter_=g_filter_[g_filter_["general_rank_STRUCTURED"].isin(["A","B","S","L","N","F"])]

print(g_filter.describe())


        original_id        target3        target6            id  \
count  8.694380e+05  869438.000000  869438.000000  8.694380e+05   
mean   2.083029e+07       0.007363       0.013388  2.083029e+07   
std    1.199797e+07       0.085494       0.114929  1.199797e+07   
min    1.000001e+06       0.000000       0.000000  1.000001e+06   
25%    1.303462e+07       0.000000       0.000000  1.303462e+07   
50%    2.000145e+07       0.000000       0.000000  2.000145e+07   
75%    2.801496e+07       0.000000       0.000000  2.801496e+07   
max    4.703118e+07       1.000000       1.000000  4.703118e+07   

       northlatitude  eastlongitude  net_reservation_flg  \
count  868633.000000  868633.000000        869438.000000   
mean       35.507799     137.054747             0.082691   
std         2.559043       3.995540             0.275415   
min         0.000000       0.000000             0.000000   
25%        34.673365     135.416475             0.000000   
50%        35.437477     138.207717 

In [59]:
ggoogle=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_GOOGLE"].value_counts()

A    380737
L    281879
S    166763
B     26439
N     11274
F      2346
Name: general_rank_GOOGLE, dtype: int64

In [ ]:
ggoogle=pseudo_time_series_filter[pseudo_time_series_filter["general_rank_GOOGLE"].isin(["A","B","S","L","N","F"])]["general_rank_GOOGLE"].value_counts()

In [120]:
rank_num=pd.DataFrame()
for column in general_rank_columns:
    print(column,pseudo_time_series_filter[column].dropna().shape[0])
    #上記を転置してrank_numに追加
    rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])

general_rank_GOOGLE 869438
general_rank_RETTY 637495


/tmp/ipykernel_8948/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


general_rank_HOTPEPPER 517620
general_rank_STRUCTURED 294864


/tmp/ipykernel_8948/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])
/tmp/ipykernel_8948/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


general_rank_TRIPADVISOR 413841
general_rank_HITOSARA 633110


/tmp/ipykernel_8948/3118007309.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rank_num=rank_num.append([[column,pseudo_time_series_filter[column].dropna().shape[0]]])


In [131]:
rank_num.sort_values(by=1,ascending=False)

,0,1
0,general_rank_GOOGLE,869438
0,general_rank_RETTY,637495
0,general_rank_HITOSARA,633110
0,general_rank_HOTPEPPER,517620
0,general_rank_TRIPADVISOR,413841
0,general_rank_STRUCTURED,294864


In [122]:
rank_num.sort_values(by=1,ascending=False)[0].values

array(['general_rank_GOOGLE', 'general_rank_RETTY',
       'general_rank_HITOSARA', 'general_rank_HOTPEPPER',
       'general_rank_TRIPADVISOR', 'general_rank_STRUCTURED'],
      dtype=object)

In [129]:
g_filter=pseudo_time_series_filter
rlist=["A","B","S","L","N","F"]
rlist=["A","B","S"]
num_restaurants=pd.DataFrame()
for column in rank_num.sort_values(by=1,ascending=False)[0].values:
    if(column=="general_rank_STRUCTURED"):
        continue
    g_filter=g_filter[g_filter[column].isin(rlist)]
    num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])

/tmp/ipykernel_8948/2945448064.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_8948/2945448064.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])
/tmp/ipykernel_8948/2945448064.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  num_restaurants=num_restaurants.append([[column,g_filter[g_filter[column].isin(rlist)][column].shape[0]]])


In [130]:
num_restaurants

,0,1
0,general_rank_GOOGLE,573939
0,general_rank_RETTY,354547
0,general_rank_HITOSARA,291669
0,general_rank_HOTPEPPER,166415
0,general_rank_TRIPADVISOR,81058
